In [1]:
from Bio import  SeqIO
from Bio.Align import PairwiseAligner
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from Bio import AlignIO
from scipy.stats import fisher_exact
print("All packages loaded successfully")

All packages loaded successfully


# Load the Sequences

Load the reference genome

In [2]:
ref_file = "reference.fasta"
ref_seq = SeqIO.read(ref_file, "fasta")
print("Reference Sequence ID:", ref_seq.id)
print(f"Reference length: {len(ref_seq.seq)}")

Reference Sequence ID: lcl|NC_045512.2_cds_YP_009724389.1_1
Reference length: 21291


Load the strain sequences

In [3]:
strain1_file = "Strain1.fasta"
strain1_seq = SeqIO.read(strain1_file, "fasta")
print("Strain1 Sequence ID:", strain1_seq.id)
print(f"Strain1 length: {len(strain1_seq.seq)}")
strain2_file = "Strain2.fasta"
strain2_seq = SeqIO.read(strain2_file, "fasta")
print("Strain2 Sequence ID:", strain2_seq.id)
print(f"Strain2 length: {len(strain2_seq.seq)}")

Strain1 Sequence ID: lcl|MN908947.3_cds_QHD43415.1_1
Strain1 length: 21291
Strain2 Sequence ID: lcl|MT034054.1_cds_QHX41666.1_1
Strain2 length: 21291


# Count Nucleotides & GC content

In [4]:
# Reference Sequence
nuc_counts_ref = pd.Series(dict(A=ref_seq.seq.count('A'),
                                T=ref_seq.seq.count('T'),
                                G=ref_seq.seq.count('G'),
                                C=ref_seq.seq.count('C')))
gc_ref = (nuc_counts_ref['G'] + nuc_counts_ref['C']) / len(ref_seq.seq) * 100
print(f"Nucleotide Counts, Reference Sequence:", nuc_counts_ref)
print(f"GC Content, Reference Sequence:, {gc_ref:2f}%")
# Strain 1
nuc_counts_strain1 = pd.Series(dict(A=strain1_seq.seq.count('A'),
                                T=strain1_seq.seq.count('T'),
                                G=strain1_seq.seq.count('G'),
                                C=strain1_seq.seq.count('C')))
gc_strain1 = (nuc_counts_strain1['G'] + nuc_counts_strain1['C']) / len(strain1_seq.seq) * 100
print(f"Nucleotide Counts, Strain1 Sequence:", nuc_counts_strain1)
print(f"GC Content, Strain1 Sequence:, {gc_strain1:2f}%")
# Strain 2
nuc_counts_strain2 = pd.Series(dict(A=strain2_seq.seq.count('A'),
                                T=strain2_seq.seq.count('T'),
                                G=strain2_seq.seq.count('G'),
                                C=strain2_seq.seq.count('C')))
gc_strain2 = (nuc_counts_strain2['G'] + nuc_counts_strain2['C']) / len(strain2_seq.seq) * 100
print(f"Nucleotide Counts, Strain2 Sequence:", nuc_counts_strain2)
print(f"GC Content, Strain2 Sequence:, {gc_strain2:2f}%")

Nucleotide Counts, Reference Sequence: A    6425
T    6891
G    4230
C    3745
dtype: int64
GC Content, Reference Sequence:, 37.457142%
Nucleotide Counts, Strain1 Sequence: A    6425
T    6891
G    4230
C    3745
dtype: int64
GC Content, Strain1 Sequence:, 37.457142%
Nucleotide Counts, Strain2 Sequence: A    6425
T    6890
G    4230
C    3746
dtype: int64
GC Content, Strain2 Sequence:, 37.461838%


# Pairwise Alignment

In [5]:
# initialize aligner
aligner = PairwiseAligner()
aligner.mode = 'global'
aligner.match_score = 1
aligner.mismatch_score = -1
aligner.open_gap_score = -2
aligner.extend_gap_score = -0.5

# Mutation Table Function

In [6]:
def get_mutations(ref_seq, query_seq):
    mutations = []
    for i, (a, b) in enumerate(zip(ref_seq, query_seq)):
        if a != b and a != "-" and b != "-":
            mutations.append((i, a, b))
    return mutations

pairwise_results = {}

Compare REFERENCE vs STRAIN1

In [7]:
alignment_ref_s1 = aligner.align(ref_seq, strain1_seq)
print("Reference vs Strain1 Alignment:")
print(alignment_ref_s1[0])
pairwise_results['Ref_vs_Strain1'] = get_mutations(ref_seq, strain1_seq)

Reference vs Strain1 Alignment:
lcl|NC_04         0 ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTT
                  0 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MN908         0 ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTT

lcl|NC_04        60 TTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCA
                 60 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MN908        60 TTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCA

lcl|NC_04       120 GAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTT
                120 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MN908       120 GAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTT

lcl|NC_04       180 TTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTGCACCT
                180 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MN908       180 TTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACT

Compare REFERENCE vs STRAIN2

In [8]:
alignment_ref_s2 = aligner.align(ref_seq, strain2_seq)
print("Reference vs Strain2 Alignment:")
print(alignment_ref_s2[0])
pairwise_results['Ref_vs_Strain2'] = get_mutations(ref_seq, strain2_seq)

Reference vs Strain2 Alignment:
lcl|NC_04         0 ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTT
                  0 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MT034         0 ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTT

lcl|NC_04        60 TTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCA
                 60 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MT034        60 TTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCA

lcl|NC_04       120 GAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTT
                120 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MT034       120 GAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTT

lcl|NC_04       180 TTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTGCACCT
                180 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MT034       180 TTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACT

Compare STRAIN1 vs STRAIN2

In [9]:
alignment_s1_s2 = aligner.align(strain1_seq, strain2_seq)
print("Strain1 vs Strain2 Alignment:")
print(alignment_s1_s2[0])
pairwise_results['Strain1_vs_Strain2'] = get_mutations(strain1_seq, strain2_seq)

Strain1 vs Strain2 Alignment:
lcl|MN908         0 ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTT
                  0 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MT034         0 ATGGAGAGCCTTGTCCCTGGTTTCAACGAGAAAACACACGTCCAACTCAGTTTGCCTGTT

lcl|MN908        60 TTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCA
                 60 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MT034        60 TTACAGGTTCGCGACGTGCTCGTACGTGGCTTTGGAGACTCCGTGGAGGAGGTCTTATCA

lcl|MN908       120 GAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTT
                120 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MT034       120 GAGGCACGTCAACATCTTAAAGATGGCACTTGTGGCTTAGTAGAAGTTGAAAAAGGCGTT

lcl|MN908       180 TTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTGCACCT
                180 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
lcl|MT034       180 TTGCCTCAACTTGAACAGCCCTATGTGTTCATCAAACGTTCGGATGCTCGAACTGC

# Multiple Sequence Alignment

In [116]:
msa_file = "MSA.aln-fasta"
aligned_seqs = list(SeqIO.parse(msa_file, "fasta"))
for record in aligned_seqs:
    print(record.id, len(record.seq))
# Assign Sequence ID
reference = aligned_seqs[0]
strain1 = aligned_seqs[1]
strain2 = aligned_seqs[2]

lcl|NC_045512.2_cds_YP_009724389.1_1 21291
lcl|MN908947.3_cds_QHD43415.1_1 21291
lcl|MT034054.1_cds_QHX41666.1_1 21291


# Mutations

In [117]:
def get_mutations(ref_seq, query_seq):
    """Returns a list of mutation dictionaries for ref vs query"""
    mutations = []
    for i, (r_base, q_base) in enumerate(zip(ref_seq.seq, query_seq.seq)):
        if r_base != q_base:
            mutations.append({
                "Position": i + 1, # 1-based indexing
                "Reference": r_base,
                "Query": q_base
            })
    return mutations
# Get mutations for each strain vs reference
mutations_s1 = get_mutations(reference, strain1)
mutations_s2 = get_mutations(reference, strain2)

# Combine into DataFrame
mut_df_s1 = pd.DataFrame(mutations_s1)
mut_df_s2 = pd.DataFrame(mutations_s2)

# Add strain label
mut_df_s1["Strain"] = "Strain1"
mut_df_s2["Strain"] = "Strain2"

mutations_df = pd.concat([mut_df_s1, mut_df_s2], ignore_index=True)
print("First few mutations:")
display(mutations_df.head())

First few mutations:


,Strain,Position,Reference,Query
0,Strain2,13005.0,T,C


# Mutation Matrix


In [119]:
alignment = AlignIO.read("MSA.aln-fasta", "fasta")

seq_ids = [record.id for record in alignment]
seqs = [str(record.seq) for record in alignment]

ref = seqs[0]
strain1 = seqs[1]
strain2 = seqs[2]

length = len(ref)
print("Alignment length:", length)

data = []

for i in range(length):
    r = ref[i]
    s1 = strain1[i]
    s2 = strain2[i]

    if r == "-":
        continue

    data.append({
        "Position": i + 1,
        "Ref": r,
        "Strain1": s1,
        "Strain2": s2,
        "Mut_S1": int(s1 != r),
        "Mut_S2": int(s2 != r)
    })
df = pd.DataFrame(data)
df.head()
# DE Style Statistics
results = []

for _, row in df.iterrows():
    table = [
        [row["Mut_S1"], row["Mut_S2"]],
        [1 - row["Mut_S1"], 1 - row["Mut_S2"]]
    ]

    _, p = fisher_exact(table)

    log2fc = np.log2((row["Mut_S2"] + 1) / (row["Mut_S1"] + 1))

    results.append({
        "Position": row["Position"],
        "log2FC": log2fc,
        "pvalue": p
    })

de_df = pd.DataFrame(results)
de_df["neglog10_p"] = -np.log10(de_df["pvalue"] + 1e-10)

de_df.head()

Alignment length: 21291


,Position,log2FC,pvalue,neglog10_p
0,1,0.0,1.0,-4.342945e-11
1,2,0.0,1.0,-4.342945e-11
2,3,0.0,1.0,-4.342945e-11
3,4,0.0,1.0,-4.342945e-11
4,5,0.0,1.0,-4.342945e-11
